In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import zscore

In [2]:
# read in single cell TPM counts
sc_df = pd.read_csv("../data/single_cell_tpm.tsv",sep="\t",index_col=0)
sc_df

,SAMEA104470928,SAMEA104470919,SAMEA104470925,SAMEA104470893,SAMEA104470910,SAMEA104470942,SAMEA104470914,SAMEA104470927,SAMEA104470918,SAMEA104470899,...,SAMEA104470912,SAMEA104470909,SAMEA104470917,SAMEA104470923,SAMEA104470938,SAMEA104470920,SAMEA104470926,SAMEA104470939,SAMEA104470955,SAMEA104470904
ENSMUSG00000000001,35.043289,14.164700,33.004907,11.756451,0.000000,15.243614,16.576417,19.673177,6.181778,24.444860,...,34.492979,17.826599,5.814934,22.814997,19.229322,23.816354,26.147754,15.384410,19.263033,27.237686
ENSMUSG00000000028,37.954773,32.130392,35.816060,53.874059,31.815447,13.876921,39.484105,52.842060,40.773227,29.572970,...,17.930434,18.896744,93.797428,30.722752,33.335661,0.000000,23.015541,20.679796,66.251897,59.144044
ENSMUSG00000000037,0.000000,0.000000,0.000000,0.642032,0.000000,17.078370,3.556361,8.493496,42.760867,0.000000,...,0.000000,1.623135,0.000000,2.209541,4.118330,0.000000,1.544047,5.762653,0.000000,0.000000
ENSMUSG00000000056,4.757336,0.000000,0.179547,0.931422,0.000000,0.000000,2.149731,0.000000,2.693681,10.651734,...,0.000000,3.932846,0.000000,5.457098,2.083688,4.792990,3.567816,1.862135,13.580021,14.642312
ENSMUSG00000000058,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,11.866426,...,1.833748,0.000000,0.000000,0.000000,0.000000,42.957469,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSMUSG00000118669,19.139697,73.581641,30.468068,87.715263,0.000000,48.693932,14.523746,13.347262,31.229248,40.787727,...,24.886516,31.052481,0.000000,48.288681,42.901008,0.221709,27.539560,46.256891,37.697013,16.006566
ENSMUSG00000118670,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.659361,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ENSMUSG00000118671,0.590141,0.000000,0.217009,0.000000,3.045656,0.000000,0.000000,1.521217,0.617970,0.000000,...,1.409446,0.272398,0.000000,0.677727,0.400472,0.000000,0.294518,2.555997,0.504701,0.363128
ENSMUSG00000118672,0.000000,0.000000,0.000000,0.000000,0.000000,6.431573,0.000000,0.000000,3.912949,0.000000,...,0.000000,0.000000,0.000000,0.000000,2.137867,0.000000,0.000000,0.000000,0.000000,0.000000


In [3]:
# filter out genes with 0 TPM in all cells
sc_df_filtered = sc_df.loc[np.sum(sc_df,axis=1)!=0] # -743
sc_df_filtered

,SAMEA104470928,SAMEA104470919,SAMEA104470925,SAMEA104470893,SAMEA104470910,SAMEA104470942,SAMEA104470914,SAMEA104470927,SAMEA104470918,SAMEA104470899,...,SAMEA104470912,SAMEA104470909,SAMEA104470917,SAMEA104470923,SAMEA104470938,SAMEA104470920,SAMEA104470926,SAMEA104470939,SAMEA104470955,SAMEA104470904
ENSMUSG00000000001,35.043289,14.164700,33.004907,11.756451,0.000000,15.243614,16.576417,19.673177,6.181778,24.444860,...,34.492979,17.826599,5.814934,22.814997,19.229322,23.816354,26.147754,15.384410,19.263033,27.237686
ENSMUSG00000000028,37.954773,32.130392,35.816060,53.874059,31.815447,13.876921,39.484105,52.842060,40.773227,29.572970,...,17.930434,18.896744,93.797428,30.722752,33.335661,0.000000,23.015541,20.679796,66.251897,59.144044
ENSMUSG00000000037,0.000000,0.000000,0.000000,0.642032,0.000000,17.078370,3.556361,8.493496,42.760867,0.000000,...,0.000000,1.623135,0.000000,2.209541,4.118330,0.000000,1.544047,5.762653,0.000000,0.000000
ENSMUSG00000000056,4.757336,0.000000,0.179547,0.931422,0.000000,0.000000,2.149731,0.000000,2.693681,10.651734,...,0.000000,3.932846,0.000000,5.457098,2.083688,4.792990,3.567816,1.862135,13.580021,14.642312
ENSMUSG00000000058,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,11.866426,...,1.833748,0.000000,0.000000,0.000000,0.000000,42.957469,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSMUSG00000118669,19.139697,73.581641,30.468068,87.715263,0.000000,48.693932,14.523746,13.347262,31.229248,40.787727,...,24.886516,31.052481,0.000000,48.288681,42.901008,0.221709,27.539560,46.256891,37.697013,16.006566
ENSMUSG00000118670,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.659361,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ENSMUSG00000118671,0.590141,0.000000,0.217009,0.000000,3.045656,0.000000,0.000000,1.521217,0.617970,0.000000,...,1.409446,0.272398,0.000000,0.677727,0.400472,0.000000,0.294518,2.555997,0.504701,0.363128
ENSMUSG00000118672,0.000000,0.000000,0.000000,0.000000,0.000000,6.431573,0.000000,0.000000,3.912949,0.000000,...,0.000000,0.000000,0.000000,0.000000,2.137867,0.000000,0.000000,0.000000,0.000000,0.000000


In [4]:
# read in gene location information
gloc = pd.read_csv("../data/gene_locations.tsv",sep="\t",index_col=0)
gloc

,seqname,start,end,gene_name
gene_id,,,,
ENSMUSG00000102693,chr1,3073253,3074322,4933401J01Rik
ENSMUSG00000064842,chr1,3102016,3102125,Gm26206
ENSMUSG00000051951,chr1,3205901,3671498,Xkr4
ENSMUSG00000102851,chr1,3252757,3253236,Gm18956
ENSMUSG00000103377,chr1,3365731,3368549,Gm37180
...,...,...,...,...
ENSMUSG00000064368,chrM,13552,14070,mt-Nd6
ENSMUSG00000064369,chrM,14071,14139,mt-Te
ENSMUSG00000064370,chrM,14145,15288,mt-Cytb


In [5]:
excl_chrom = ['chrM','chrX','chrY']
set(gloc.seqname).difference(excl_chrom)

{'chr1',
 'chr10',
 'chr11',
 'chr12',
 'chr13',
 'chr14',
 'chr15',
 'chr16',
 'chr17',
 'chr18',
 'chr19',
 'chr2',
 'chr3',
 'chr4',
 'chr5',
 'chr6',
 'chr7',
 'chr8',
 'chr9'}

In [6]:
gloc_filtered = gloc[gloc['seqname'].isin(set(gloc.seqname).difference(excl_chrom))]
gloc_filtered

,seqname,start,end,gene_name
gene_id,,,,
ENSMUSG00000102693,chr1,3073253,3074322,4933401J01Rik
ENSMUSG00000064842,chr1,3102016,3102125,Gm26206
ENSMUSG00000051951,chr1,3205901,3671498,Xkr4
ENSMUSG00000102851,chr1,3252757,3253236,Gm18956
ENSMUSG00000103377,chr1,3365731,3368549,Gm37180
...,...,...,...,...
ENSMUSG00000117740,chr19,61232424,61232926,Gm4242
ENSMUSG00000117782,chr19,61255967,61258627,Gm18999
ENSMUSG00000117951,chr19,61257534,61257789,Gm50357


In [32]:
gene_list = set(sc_df_filtered.index).intersection(gloc_filtered.index)
sc_df_filtered = sc_df_filtered.loc[gene_list]
sc_df_filtered

,SAMEA104470928,SAMEA104470919,SAMEA104470925,SAMEA104470893,SAMEA104470910,SAMEA104470942,SAMEA104470914,SAMEA104470927,SAMEA104470918,SAMEA104470899,...,SAMEA104470912,SAMEA104470909,SAMEA104470917,SAMEA104470923,SAMEA104470938,SAMEA104470920,SAMEA104470926,SAMEA104470939,SAMEA104470955,SAMEA104470904
ENSMUSG00000026455,13.186207,1.688566,20.572151,5.150118,0.000000,13.001073,8.468970,8.629492,8.777136,5.134984,...,8.209745,7.767276,5.897341,8.323001,6.288733,1.084546e+00,11.557141,17.270804,7.854771,15.173553
ENSMUSG00000103304,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.382078,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000
ENSMUSG00000081543,0.377946,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.227064,0.000000e+00,1.062714,0.000000,0.000000,0.627709
ENSMUSG00000029075,0.000000,0.000000,0.000000,1.660805,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000
ENSMUSG00000021200,0.000000,0.000000,0.000000,4.121103,0.000000,3.229583,0.877989,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.689954,0.000000e+00,0.000000,3.802648,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSMUSG00000056608,11.764291,17.363674,15.235305,43.822893,30.870098,29.690992,14.740152,18.406251,12.493751,3.898264,...,31.513126,7.005787,8.640907,18.360860,12.293265,1.108407e+01,11.042692,20.490944,13.092280,9.409589
ENSMUSG00000042390,18.980139,28.108929,13.708170,5.548341,0.000000,4.199269,19.064166,44.323973,7.569968,0.000000,...,16.379763,14.589322,32.803581,14.729247,5.740154,3.739087e+00,20.409703,26.345922,5.564153,8.662832
ENSMUSG00000001249,4.086343,0.000000,24.810103,15.504089,0.000000,0.000000,21.470163,3.233532,9.937483,0.000000,...,32.489597,12.898309,23.215750,0.000000,1.326595,0.000000e+00,8.597797,0.000000,0.000000,9.568627
ENSMUSG00000112196,0.000000,0.000000,1.861059,0.000000,0.000000,0.000000,12.590257,0.000000,0.000000,0.000000,...,0.000000,0.615117,0.000000,2.726630,0.807941,0.000000e+00,2.416076,0.000000,0.000000,1.260289


In [25]:
gloc_filtered = gloc_filtered.loc[set(sc_df_filtered.index).intersection(gloc_filtered.index)]
gloc_filtered

,seqname,start,end,gene_name
gene_id,,,,
ENSMUSG00000026455,chr1,134455531,134491018,Klhl12
ENSMUSG00000103304,chr3,88474584,88475154,Gm5979
ENSMUSG00000081543,chr11,77644144,77644910,Gm12733
ENSMUSG00000029075,chr4,156013843,156016612,Tnfrsf4
ENSMUSG00000021200,chr12,103321142,103356001,Asb2
...,...,...,...,...
ENSMUSG00000056608,chr8,90828352,91054516,Chd9
ENSMUSG00000042390,chr3,90293178,90363407,Gatad2b
ENSMUSG00000001249,chr7,31098725,31115290,Hpn


In [29]:
#gloc_filtered[gloc_filtered['seqname']=='chr1']
gloc_filtered[gloc_filtered['seqname']=='chr1'].index

Index(['ENSMUSG00000026455', 'ENSMUSG00000052477', 'ENSMUSG00000023150',
       'ENSMUSG00000101945', 'ENSMUSG00000056055', 'ENSMUSG00000047496',
       'ENSMUSG00000039748', 'ENSMUSG00000048000', 'ENSMUSG00000067064',
       'ENSMUSG00000034432',
       ...
       'ENSMUSG00000073725', 'ENSMUSG00000084333', 'ENSMUSG00000053286',
       'ENSMUSG00000033722', 'ENSMUSG00000042215', 'ENSMUSG00000100702',
       'ENSMUSG00000025967', 'ENSMUSG00000026142', 'ENSMUSG00000089888',
       'ENSMUSG00000026604'],
      dtype='object', name='gene_id', length=1423)

In [31]:
sc_df_filtered.loc[gloc_filtered[gloc_filtered['seqname']=='chr1'].index]

,SAMEA104470928,SAMEA104470919,SAMEA104470925,SAMEA104470893,SAMEA104470910,SAMEA104470942,SAMEA104470914,SAMEA104470927,SAMEA104470918,SAMEA104470899,...,SAMEA104470912,SAMEA104470909,SAMEA104470917,SAMEA104470923,SAMEA104470938,SAMEA104470920,SAMEA104470926,SAMEA104470939,SAMEA104470955,SAMEA104470904
gene_id,,,,,,,,,,,,,,,,,,,,,
ENSMUSG00000026455,13.186207,1.688566,20.572151,5.150118,0.000000,13.001073,8.468970,8.629492,8.777136,5.134984,...,8.209745,7.767276,5.897341,8.323001,6.288733,1.084546,11.557141,17.270804,7.854771,15.173553
ENSMUSG00000052477,0.000000,0.000000,1.024634,0.307366,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,12.679952,0.000000,0.000000,0.000000,0.000000
ENSMUSG00000023150,45.139651,36.291501,28.948138,36.331505,14.512622,42.495340,62.326409,39.465667,37.148660,42.929544,...,65.151463,37.053571,28.569669,40.564122,42.097134,11.280542,60.825911,55.036342,48.686553,69.420685
ENSMUSG00000101945,2.238337,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,1.592289,0.989872,0.000000,0.000000,0.000000,0.000000,0.000000
ENSMUSG00000056055,40.356775,35.589343,4.123370,9.078941,135.463040,16.641110,23.987551,35.670324,65.393816,18.611181,...,28.784805,21.984862,35.732570,31.337896,52.530521,12.987291,24.264494,31.545355,12.342244,5.043231
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSMUSG00000100702,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ENSMUSG00000025967,1731.828606,2224.515724,2815.361002,2197.425292,1467.253717,1739.133595,2123.542098,1611.970145,1189.084715,1563.256336,...,2199.957777,1837.560344,3128.562918,2843.697880,2163.775970,573.232045,1453.997453,1675.335971,2597.867860,1500.327443
ENSMUSG00000026142,2.421633,0.000000,1.758955,2.661393,10.849784,0.000000,1.053004,1.166408,0.000000,4.347953,...,2.396559,3.655721,0.000000,0.507257,3.126060,4.848620,2.974881,4.560652,0.000000,0.486091


In [27]:
sum(sc_df_filtered.index=='ENSMUSG00000067351')

1

In [ ]:
min(gloc_filtered[gloc_filtered['seqname']=='chr1']['start'])

In [ ]:
#len(set(sc_df.index).intersection(gloc.index)) #24826 - 54 = 24772 # before filtering out chromosomes
#len(set(sc_df_filtered.index).intersection(gloc.index)) # 24083 - 38 = 24045
##set(sc_df_filtered.index) - set(gloc.index)
##set(sc_df.index) - set(gloc.index)

In [ ]:
sc_df_filtered.loc[gloc[gloc['seqname']=='chr1'].index]

In [33]:
['chr'+str(i+1) for i in range(19)]

['chr1',
 'chr2',
 'chr3',
 'chr4',
 'chr5',
 'chr6',
 'chr7',
 'chr8',
 'chr9',
 'chr10',
 'chr11',
 'chr12',
 'chr13',
 'chr14',
 'chr15',
 'chr16',
 'chr17',
 'chr18',
 'chr19']

In [34]:
zscore(np.log2(sc_df_filtered+1),axis=1)

,SAMEA104470928,SAMEA104470919,SAMEA104470925,SAMEA104470893,SAMEA104470910,SAMEA104470942,SAMEA104470914,SAMEA104470927,SAMEA104470918,SAMEA104470899,...,SAMEA104470912,SAMEA104470909,SAMEA104470917,SAMEA104470923,SAMEA104470938,SAMEA104470920,SAMEA104470926,SAMEA104470939,SAMEA104470955,SAMEA104470904
ENSMUSG00000026455,0.561814,-1.535103,1.090225,-0.491899,-2.781970,0.545253,0.052165,0.073359,0.092542,-0.495005,...,0.017170,-0.044903,-0.347338,0.032579,-0.277754,-1.855903,0.408030,0.880820,-0.032383,0.727104
ENSMUSG00000103304,-0.219635,-0.219635,-0.219635,-0.219635,-0.219635,-0.219635,-0.219635,-0.219635,-0.219635,-0.219635,...,-0.219635,2.385304,-0.219635,-0.219635,-0.219635,-0.219635,-0.219635,-0.219635,-0.219635,-0.219635
ENSMUSG00000081543,1.089802,-0.440788,-0.440788,-0.440788,-0.440788,-0.440788,-0.440788,-0.440788,-0.440788,-0.440788,...,-0.440788,-0.440788,-0.440788,-0.440788,0.536137,-0.440788,3.015863,-0.440788,-0.440788,1.885091
ENSMUSG00000029075,-0.248019,-0.248019,-0.248019,2.025323,-0.248019,-0.248019,-0.248019,-0.248019,-0.248019,-0.248019,...,-0.248019,-0.248019,-0.248019,-0.248019,-0.248019,-0.248019,-0.248019,-0.248019,-0.248019,-0.248019
ENSMUSG00000021200,-0.511754,-0.511754,-0.511754,2.284182,-0.511754,1.956780,0.566999,-0.511754,-0.511754,-0.511754,...,-0.511754,-0.511754,-0.511754,-0.511754,0.386408,-0.511754,-0.511754,2.174283,-0.511754,-0.511754
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSMUSG00000056608,-0.135191,0.365670,0.196038,1.594465,1.124824,1.072911,0.153386,0.441711,-0.058662,-1.454080,...,1.152331,-0.777562,-0.521639,0.438487,-0.079275,-0.210602,-0.215326,0.582220,0.001102,-0.416003
ENSMUSG00000042390,0.727894,1.150998,0.383463,-0.526359,-2.639269,-0.785742,0.732613,1.648854,-0.223848,-2.639269,...,0.571122,0.448882,1.319114,0.458928,-0.493898,-0.889940,0.805594,1.080751,-0.523648,-0.088899
ENSMUSG00000001249,-0.164093,-1.347439,1.017541,0.692227,-1.347439,-1.347439,0.916723,-0.297609,0.392918,-1.347439,...,1.207035,0.567210,0.971152,-1.347439,-0.733121,-1.347439,0.297860,-1.347439,-1.347439,0.367960
ENSMUSG00000112196,-0.532944,-0.532944,1.357931,-0.532944,-0.532944,-0.532944,4.160738,-0.532944,-0.532944,-0.532944,...,-0.532944,0.329410,-0.532944,1.833374,0.532280,-0.532944,1.676858,-0.532944,-0.532944,0.933957


In [38]:
tad_locs = pd.read_csv("../data/TAD_strong_boundary_start_end.csv")
tad_locs

,chrom,start,end
0,chr1,3185000.0,3225000.0
1,chr1,3225000.0,3295000.0
2,chr1,3295000.0,3545000.0
3,chr1,3545000.0,3665000.0
4,chr1,3665000.0,3865000.0
...,...,...,...
16977,chrX,169585000.0,169675000.0
16978,chrX,169675000.0,169775000.0
16979,chrX,169775000.0,169855000.0
16980,chrX,169855000.0,169985000.0


In [39]:
excl_chrom = ['chrM','chrX','chrY']
tad_locs[tad_locs['chrom'].isin(set(gloc.seqname).difference(excl_chrom))]

,chrom,start,end
0,chr1,3185000.0,3225000.0
1,chr1,3225000.0,3295000.0
2,chr1,3295000.0,3545000.0
3,chr1,3545000.0,3665000.0
4,chr1,3665000.0,3865000.0
...,...,...,...
15859,chr19,60675000.0,60745000.0
15860,chr19,60745000.0,60895000.0
15861,chr19,60895000.0,61095000.0
15862,chr19,61095000.0,61265000.0
